In [13]:
import os
from subprocess import Popen, PIPE
import sys

In [15]:
PRAAT= '/usr/bin/praat'

def run_praat(script_file):
    args= [PRAAT, '--run', script_file]
    Popen(args, stdout=PIPE)

Lista de archivos de audio - concatenar audio y escribir a archivo

In [32]:
def output_audio(filenames, output_name):

    script = ""
    for i, f in enumerate(filenames):
        script += "Read from file: \"" + f + "\"\n"
        script += "selectObject: \"Sound " + f[-6:-4] + "\"\n"
        script += "Rename: \"" + f[-6:-4] + str(i) + "\"\n"

    script += "selectObject: \"Sound " + filenames[0][-6:-4] + str(0) + "\"\n"
    for i, f in enumerate(filenames[1:]):
        script += "plusObject: \"Sound " + f[-6:-4] + str(i+1) + "\"\n"

    script += "Concatenate recoverably\n"
    script += "selectObject: \"Sound chain\"\n"
    script += "Write to WAV file: \"" + output_name + "\"\n"
    script += "Save as text file: \"" + output_name + ".TextGrid\"\n"

    with open("concatenar.praat", "w") as cf:
        cf.write(script)

    run_praat("concatenar.praat")
    os.remove("concatenar.praat")

print(output_audio(["./Difonos/re.wav", "./Difonos/rE.wav"], "./file.wav"))

None


In [33]:
diphones = ["-e", "-E", "-f", "-k", "-m", "-s", "-t", "e-", "E-",
           "s-", "ee", "eE", "Ee", "EE", "fe", "fE", "ke", "kE", 
           "me", "mE", "se", "sE", "te", "tE", "re", "rE", "ef",
           "Ef", "ek", "Ek", "em", "Em", "es", "Es", "et", "Et",
           "er", "Er", "fr", "kr", "tr", "sf", "sk", "st", "ss",
           "sm"]
prefixname = "/home/user/ph/"
wavsufix = ".wav"

def diphone_list( word ):
    diphone_list = []
    word  = "-" + word + "-"
    for i, _ in enumerate(word[:-1]):
        diphone = word[i:i+2]
        if not diphone in diphones:
            print("ERROR: invalid word. Try again.")
            sys.exit(1)
        diphone_list.append(diphone)

    # agregar prefijo y sufijo
    return list(map(lambda s: prefixname + s + wavsufix, diphone_list))

diphone_list("erefes")

['/home/user/ph/-e.wav',
 '/home/user/ph/er.wav',
 '/home/user/ph/re.wav',
 '/home/user/ph/ef.wav',
 '/home/user/ph/fe.wav',
 '/home/user/ph/es.wav',
 '/home/user/ph/s-.wav']